# Impact of cut off points on transfer outcomes

We need to decide when we should consistently cut off the data to show a full picture of a month. The purpose of this analysis is to assess the impact of different cut off points. For example, if we cut off the data two weeks after the month end, we might have 1% of transfers that are shown in pending but will ultimately be successful, compared to doing it three weeks after the month end where we might ave 0.5% of transfers that are pending but ultimately successful. 

In [ ]:
import paths
from datetime import datetime
from dateutil.tz import tzutc
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
from gp2gp.date.range import DateTimeRange
from gp2gp.pipeline.dashboard.main import read_spine_csv_gz_files
from gp2gp.service.transformers import derive_transfers
from scripts.gp2gp_spine_outcomes import parse_conversations

In [ ]:
july_data_file_name = "../data/months/July-2020.csv.gz"
august_data_file_name = "../data/months/Aug-2020.csv.gz"
september_data_file_name="../data/months/Sept-2020.csv.gz"
october_data_file_name = "../data/months/Oct-2020.csv.gz"

In [ ]:
july_time_range = DateTimeRange(
    datetime(year=2020, month=7, day=1, tzinfo=tzutc()),
    datetime(year=2020, month=8, day=1, tzinfo=tzutc()),
)

In [ ]:
  spine_messages = read_spine_csv_gz_files([
      july_data_file_name, august_data_file_name
  ])

  conversations = parse_conversations(spine_messages, time_range=july_time_range)
  transfers = derive_transfers(conversations)

In [ ]:
transfers_df = pd.DataFrame(transfers)[["status", "date_completed"]]

transfers_df

## Investigation into the final outcome of transfers with intermediate errors

In [ ]:
  spine_messages = read_spine_csv_gz_files([
      july_data_file_name, august_data_file_name, september_data_file_name, october_data_file_name
  ])

  conversations = parse_conversations(spine_messages, time_range=july_time_range)
  transfers = derive_transfers(conversations)

In [ ]:
# Below is taken from PRMT-1408 GP2GP Technical Error Counts
total_number_transfers = 165069

def filter_transfers_with_intermediate_errors(transfer):
    return len(transfer.intermediate_error_codes) > 0

transfers_with_intermediate_errors = filter(filter_transfers_with_intermediate_errors, transfers)

In [ ]:
transfers_with_intermediate_errors_df = pd.DataFrame(transfers_with_intermediate_errors)[["final_error_code", "intermediate_error_codes", "status", "date_completed",]]
transfers_with_intermediate_errors_df

In [ ]:
print(f"Total number of transfers: {total_number_transfers}")

In [ ]:
def calculate_percentage(subset, total):
    return round((subset/total) * 100, 2)

In [ ]:
total_number_transfers_with_intermediate_error_codes = len(transfers_with_intermediate_errors_df) - 1

print(f"Total number of transfers with intermediate errors: {total_number_transfers_with_intermediate_error_codes}, out of a total transfer count of {total_number_transfers} ({calculate_percentage(total_number_transfers_with_intermediate_error_codes, total_number_transfers)}%) for transfers that began in the month of July (and cut off point being up until October)")

In [ ]:
eventually_integrated_count = transfers_with_intermediate_errors_df["date_completed"].describe()["count"]
percentage_eventually_integrated = calculate_percentage(eventually_integrated_count, total_number_transfers_with_intermediate_error_codes)

print(f"Eventually integrated: {eventually_integrated_count} out of {total_number_transfers_with_intermediate_error_codes} ({percentage_eventually_integrated}%) messages with intermediate error codes")

In [ ]:
eventually_fails_count = transfers_with_intermediate_errors_df["final_error_code"].describe()["count"]
percentage_eventually_failed = calculate_percentage(eventually_fails_count, total_number_transfers_with_intermediate_error_codes)

print(f"Eventually failed: {eventually_fails_count} out of {total_number_transfers_with_intermediate_error_codes} ({percentage_eventually_failed}%) messages with intermediate error codes")

In [ ]:
unresolved_count = total_number_transfers_with_intermediate_error_codes - (eventually_fails_count + eventually_integrated_count)
percentage_unresolved = calculate_percentage(still_pending, total_number_transfers_with_intermediate_error_codes)

print(f"Still pending: {still_pending} out of {total_number_transfers_with_intermediate_error_codes} ({percentage_unresolved})% messages with intermediate error codes")